In [1]:
import pandas as pd
import numpy as np
import re
import os
from collections import Counter

Collect the list of txt files for each play and save to spPlays

In [2]:
#use this list to exclude certain plays
excludedPlays = []

spDir = 'shakespeare-plays-plus'
spSubDir = [spDir + '/'+ x for x in os.listdir(spDir) if '.' not in x]
spPlays = []

for subDir in spSubDir:
    spPlays = spPlays + [subDir + '/' + x for x in os.listdir(subDir) if 'txt' in x and x not in excludedPlays]


Read all selected plays  
Skip line if a line contains < (non-text)  
Create CounterDict to store a word Counter for each play  
The keys for CounterDict are play names without '.txt'

In [3]:
CounterDict = {}

for spPlay in spPlays:
    playName = spPlay.split('/')[-1].split('.')[0]
    CounterDict[playName] = Counter()
    with open(spPlay, 'r', encoding='utf-16') as playTxt:
        for line in playTxt:
            if '<' in line:
                continue
            if line.split():
                #no additional text processing other than converting everything to lower case
                CounterDict[playName].update(Counter(re.findall('\w+', line.lower())))


To do list:
* leave one word out -- we can "cheat" and check this one once we have counts of words with 1, 2, 3...appearances
* leave one play out
* randomly leave out a number of words--simulate multiple times and we should be able to plot emperical bayes estimate and error range

In [4]:
#create a totalCounter for all plays
totalCounter = sum([playCounter for playCounter in CounterDict.values()], Counter())
print('distinct words:', len(totalCounter))
print('total words:', sum(totalCounter.values()))

distinct words: 22859
total words: 822249


we have fewer words than CASI example  
probably because we are using only play data

### Leave one play out

In [5]:
for playName in CounterDict.keys():
    print(playName)
    #create hold one out counter
    hooCounter = totalCounter - CounterDict[playName]
    
    #use set difference to calculate actual number of "new words" in a play
    print('actual new words', len(set(CounterDict[playName].keys()) - set(hooCounter.keys())))
    
    #use "corbet formula" to calculate expected number of "new words" in a play
    t = sum(CounterDict[playName].values())/sum(hooCounter.values())
    expectedWords = sum(v*(t**k)*((-1)**(k-1)) for k, v in Counter(hooCounter.values()).items())
    print('expected new words', round(expectedWords))
    
    print('*'*50)

The Tempest
actual new words 217
expected new words 167
**************************************************
The Taming of the Shrew
actual new words 237
expected new words 217
**************************************************
Macbeth
actual new words 238
expected new words 175
**************************************************
Anthony and Cleopatra
actual new words 346
expected new words 256
**************************************************
The First Part of King Henry IV
actual new words 282
expected new words 250
**************************************************
Measure for Measure
actual new words 220
expected new words 224
**************************************************
Romeo And Juliet
actual new words 228
expected new words 258
**************************************************
The Second Part of King Henry IV
actual new words 313
expected new words 264
**************************************************
The Third Part of King Henry VI
actual new words 186
expected new words 